# Biolink Metamodel Test Notebook

In [1]:
!pip install yamlmagic
%reload_ext yamlmagic

In [6]:
from IPython.core.display import display, HTML

from biolinkml.meta import SchemaDefinition
from biolinkml.utils.schemaloader import SchemaLoader
from biolinkml.utils.yamlutils import DupCheckYamlLoader
from yaml import dump

from biolinkml.generators.shexgen import ShExGenerator
from biolinkml.generators.pythongen import PythonGenerator
from biolinkml.generators.yumlgen import YumlGenerator
from biolinkml.generators.jsonldcontextgen import ContextGenerator

## Basic model structure
A biolink model consists of:
* a name
* a uri
* type definitions
* slot definitions
* class definitions
* subset definitions

As an example, the model below defines:
1) 

In [14]:
%%yaml --loader DupCheckYamlLoader yaml
id: http://example.org/sample/example1
name: synopsis2
prefixes:
    foaf: http://xmlns.com/foaf/0.1/

default_curi_maps:
    - semweb_context
    
default_range: string

types:
    string:
        base: str
        uri: xsd:string
    int:
        base: int
        uri: xsd:integer
    boolean:
        base: Bool
        uri: xsd:boolean
        

classes:
    person:
        description: A person, living or dead
        slots:
            - first name
            - last name
            - age
            - living
            - knows

slots:

    first name:
        description: The first name of a person
        slot_uri: foaf:firstName
        multivalued: true
        
    last name:
        description: The last name of a person
        slot_uri: foaf:lastName
        required: true
        
    living:
        description: Whether the person is alive
        range: boolean
        comments:
            - unspecified means unknown
        
    age:
        description: The age of a person if living or age of death if not
        range: int
        slot_uri: foaf:age
        
    knows:
        description: A person known by this person (indicating some level of reciprocated interaction between the parties).
        range: person
        slot_uri: foaf:knows
        multivalued: true

<IPython.core.display.Javascript object>

### We can emit this model as a Python class

In [9]:
print(PythonGenerator(yaml).serialize())

# Auto generated from None by pythongen.py version: 0.2.0
# Generation date: 2019-03-01 16:32
# Schema: synopsis2
#
# id: http://example.org/sample/example1
# description:
# license:

from typing import Optional, List, Union, Dict
from dataclasses import dataclass
from biolinkml.utils.metamodelcore import empty_list, empty_dict
from biolinkml.utils.yamlutils import YAMLRoot
from biolinkml.utils.metamodelcore import Bool

metamodel_version = "1.0.2"

inherited_slots: List[str] = []


# Types
class String(str):
    pass


class Int(int):
    pass


class Boolean(Bool):
    pass


# Class references



@dataclass
class Person(YAMLRoot):
    """
    A person, living or dead
    """

    # === person ===
    last_name: str
    first_name: List[str] = empty_list()
    age: Optional[int] = None
    living: Optional[Bool] = None
    knows: List[Union[dict, "Person"]] = empty_list()

    def _fix_elements(self):
        super()._fix_elements()
        self.knows = [v if isinstance(v, Person)
  

### We can emit a UML rendering of  this model

In [10]:
display(HTML(f'<img src="{YumlGenerator(yaml).serialize()}"/>'))

**_note_**: `knows` is a containment relationship because we have not assigned an identity (key) to the `person` class

### We can emit a JSON-LD context for the model:

In [15]:
print(ContextGenerator(yaml).serialize())

{
   "comments": "Auto generated from None by jsonldcontextgen.py version: 0.0.2\nGeneration date: 2019-03-01 16:33\nSchema: synopsis2\n\nid: http://example.org/sample/example1\ndescription: \nlicense: \n",
   "@context": {
      "foaf": "http://xmlns.com/foaf/0.1/",
      "@vocab": "http://example.org/sample/example1",
      "age": {
         "@type": "xsd:integer",
         "@id": "foaf:age"
      },
      "first_name": {
         "@id": "foaf:firstName",
         "@container": "@list"
      },
      "knows": {
         "@container": "@list"
      },
      "last_name": {
         "@id": "foaf:lastName"
      },
      "living": {
         "@type": "xsd:boolean"
      }
   }
}



In [ ]:
### We can then use this JSON-LD context to convert model *instances* into RDF